In [1]:
import json, pickle
from tensorflow.keras.models import load_model

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

ImportError: initialization failed

In [2]:
# Reading file
intents = json.loads(open('chats.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_trained_model.h5')

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
def clean_sentence(sentence):
    s_words = nltk.word_tokenize(sentence)
    s_words = [lemmatizer.lemmatize(w) for w in s_words]
    
    return s_words

In [6]:
def bag_words(sentence):
    s_words = clean_sentence(sentence)
    bag = [0] * len(words)
    for w in s_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
                
    return np.array(bag)

In [7]:
import numpy as np
import random

In [8]:
def class_predict(sentence):
    b = bag_words(sentence)
    ans = model.predict(np.array([b]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(ans) if r > ERROR_THRESHOLD]
    
    results.sort(key = lambda x: x[1], reverse = True)
    l = []
    for r in results:
        l.append({'intent' : classes[r[0]], 'probability' : str(r[1])})
        
    return l

In [130]:
# class_predict('money')

In [9]:
stock_names = ['Apple', 'Google', "TCS"]
stock_names = '\n\t:> '.join(stock_names)

crypto_names = ["Bitcoin", 'Luna']
crypto_names = '\n\t:> '.join(crypto_names)

item = {'Potato' : 30, 'MSI Laptop' : 50000}
purchase = ''
for k,v in item.items():
    purchase += '\n\t:> ' + k + '  ₹' + str(v)

In [10]:
def response(intList, intJson):
    tag = intList[0]['intent']
    ints = intJson['intents']
    for i in ints:
        if i['tags'] == tag == 'stock':
            result = i['responses'][0] + '\t:> ' + stock_names
            break
            
        elif i['tags'] == tag == 'crypto':
            result = i['responses'][0] + '\t:> ' + crypto_names
            break
            
        elif i['tags'] == tag == 'purchase':
            result = random.choice(i['responses']) + purchase
            break
            
        elif i['tags'] == tag:
            result = random.choice(i['responses'])
            break
        
        else:
            result = 'Sorry I do not understand it'
            
    return result

In [168]:
# print(response(a, intents))

In [15]:
# Main function
print("Hello, I'm Friday!! \nYour personal assistant, How can I help you?\n")
user_name = 'Sid'

while True:
    query = input(f'{user_name} >> ')
    ints = class_predict(query)
    
    if len(ints) == 0:
        print('Firday >> Sorry I do not understand it')
        continue
        
    if ints[0]['intent'] == 'goodbye':
        ans = response(ints, intents)
        print('Firday >>',ans)
        break
        
    ans = response(ints, intents)
    print('Firday >>',ans)

Hello, I'm Friday!! 
Your personal assistant, How can I help you?

Sid >> hi
Firday >> Hey!
Sid >> bye
Firday >> Have a nice day:) Bye!
